# OUTLIER NOTEBOOK

In [2]:
import sys
sys.path.append("../notebooks/scripts/")

In [3]:
import altair as alt
from altair_saver import save
from augur.utils import json_to_tree
import json
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import re
#from reportlab.graphics import renderPDF
import seaborn as sns
#from svglib.svglib import svg2rlg

from Helpers import linking_tree_with_plots_clickable, linking_tree_with_plots_brush, scatterplot_with_tooltip_interactive
from Helpers import get_y_positions

%matplotlib inline

In [4]:
alt.renderers.set_embed_options(
    padding={"left": 0, "right": 0, "bottom": 1, "top": 1}
)

RendererRegistry.enable('default')

In [5]:
sns.set_style("ticks")
# Disable top and right spines.
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
# Display and save figures at higher resolution for presentations and manuscripts.
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['figure.dpi'] = 100
# Display text at sizes large enough for presentations and manuscripts.
mpl.rcParams['font.weight'] = "normal"
mpl.rcParams['axes.labelweight'] = "normal"
mpl.rcParams['font.size'] = 10
mpl.rcParams['axes.labelsize'] = 10
mpl.rcParams['legend.fontsize'] = 8
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
mpl.rcParams['axes.titlesize'] = 8
mpl.rc('text', usetex=False)

In [6]:
try: 
    snakemake.input.node_df
    import selenium
    from selenium.webdriver import Chrome 
    from selenium import webdriver

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument("--remote-debugging-port=9222")

    browser = webdriver.Chrome(options=chrome_options)
except:
    print("not in Snakemake, imports unnecessary")

not in Snakemake, imports unnecessary


## Pathogen-specific variables

Consider consolidating these into a single configuration file that can be passed to the notebook as a command line argument for more scriptable generation of these figures.

# Snakemake Specific Variables

In [6]:
try:
        node_df = snakemake.input.node_df,

        mds_df_human_swine = snakemake.input.mds_df_human_swine,
        mds_df_full = snakemake.input.mds_df_full,

        outlier_df_human_swine = snakemake.input.outlier_df_human_swine,
        outlier_df_full = snakemake.input.outlier_df_full,

        #KDE Density
        MDS_KDE_df_human_swine = snakemake.input.MDS_KDE_df_human_swine,
        MDS_KDE_metadata_df_human_swine = snakemake.input.MDS_KDE_metadata_df_human_swine,
        MDS_outlier_metadata_df_human_swine = snakemake.input.MDS_outlier_metadata_df_human_swine,


except:
    print("not in Snakemake, imports unnecessary")

not in Snakemake, imports unnecessary


In [7]:
domain =  ['outlier', 'not_outlier'] 
range_ = ['#FF0000','#000000']

# Reading in all the data from the scripts

In [8]:
import os

os.getcwd()

'/mnt/c/Work/BedfordProjects/cartography/outlier_analysis'

In [9]:
#node_df = pd.read_csv(node_df, sep="\t")
metadata_df = pd.read_csv("results/total_metadata.tsv", sep="\t")
node_df = pd.read_csv("results/table.tsv", sep="\t")

In [10]:
metadata_df

,strain,date,accession,country,region,outlier
0,A/Alabama/01/2017,2017-01-04,CY216617,USA,N,not_outlier
1,A/Alabama/02/2015,2015-01-26,KT836052,USA,N,not_outlier
2,A/Alabama/02/2016,2016-01-27,KX412662,USA,N,not_outlier
3,A/Alabama/02/2017,2017-01-05,CY216609,USA,N,not_outlier
4,A/Alabama/03/2017,2017-01-19,CY216625,USA,N,not_outlier
...,...,...,...,...,...,...
9573,A/swine/Virginia/A02478454/2019,2019-04-26,MK967619,USA,N,outlier
9574,A/swine/Virginia/A02478581/2019,2019-04-22,MN436839,USA,N,outlier
9575,A/swine/Virginia/A02478738/2019,2019-08-29,MN608561,USA,N,outlier
9576,A/swine/Wisconsin/A02140875/2018,2018-01-19,MG977436,USA,N,outlier


In [11]:
node_df.rename(columns={'num_date':'date', 'y_value':"y"}, inplace=True)

In [12]:
node_df.head()

,strain,date,mds1,mds2,clade_membership,mds_label,y
0,A/swine/Saskatchewan/SD0300/2018,2018.35,157.25,-160.98,unassigned,-1.0,1
1,A/swine/BritishColumbia/SD0286/2018,2018.27,196.09,-100.65,unassigned,-1.0,2
2,A/swine/Saskatchewan/SD0271/2018,2018.18,160.47,140.22,unassigned,-1.0,3
3,A/swine/Minnesota/A02245583/2020,2020.27,220.94,1.94,unassigned,-1.0,4
4,A/swine/Manitoba/D0525/2018,2018.05,106.97,182.43,unassigned,-1.0,5


In [13]:
node_df = node_df.merge(metadata_df[["strain", "outlier"]], on="strain")

# HDBSCAN Final Charts

In [14]:
MDS_df_human_swine = pd.read_csv("results/embed_mds_human_swine.csv")
MDS_df_full = pd.read_csv("results/embed_mds_full.csv")

In [15]:
node_df = pd.read_csv("results/table.tsv", sep="\t")
#node_df = pd.read_csv(node_df, sep="\t")
node_df.rename(columns={'num_date':'date', 'y_value':"y"}, inplace=True)

In [16]:
colors = pd.read_csv("../notebooks/config/color_schemes.tsv", sep="\t", names=[i for i in range(0,101)])
#colors = pd.read_csv(colors, sep="\t", names=[i for i in range(0,101)])

In [76]:
MDS_df_human_swine

,strain,mds1,mds2,mds_label,mds_label_default
0,A/swine/Iowa/A02429498/2019,159.391487,72.598835,-1,-1
1,A/swine/Missouri/A02479155/2020,73.276434,-151.097567,-1,-1
2,A/swine/Tennessee/A02428125/2018,103.878946,134.591555,-1,-1
3,A/Shiraz/1006/2015,64.402457,89.854161,-1,-1
4,A/Alabama/09/2015,3.497593,0.332706,1,19
...,...,...,...,...,...
2593,A/swine/Nebraska/A02256553/2018,95.949991,197.170133,-1,-1
2594,A/swine/Minnesota/A02245583/2020,220.937493,1.940319,-1,-1
2595,A/swine/BritishColumbia/SD0286/2018,196.087181,-100.649552,-1,-1
2596,A/swine/Saskatchewan/SD0271/2018,160.470705,140.218308,-1,-1


In [17]:
(MDS_df_human_swine["mds_label"] == -1).sum()

127

In [18]:
alt.data_transformers.disable_max_rows()
domain =  MDS_df_human_swine["mds_label"].drop_duplicates().values
range_ = colors[len(domain)-1:len(domain)].dropna(axis=1).values.tolist()[0]
chart_12_mds = scatterplot_with_tooltip_interactive(MDS_df_human_swine,'mds1','mds2','mds1','mds2',['strain', 'mds_label'],'mds_label:N', domain, range_).properties(title="mds, optimal distance threshold")

In [19]:
chart_12_mds

alt.Chart(...)

In [90]:
merged_mds_df = MDS_df_human_swine.merge(node_df[["strain", "date", "y", "clade_membership"]], on="strain")

In [91]:
merged_mds_df

,strain,mds1,mds2,mds_label,mds_label_default,date,y,clade_membership
0,A/swine/Iowa/A02429498/2019,159.391487,72.598835,-1,-1,2019.01,20,unassigned
1,A/swine/Missouri/A02479155/2020,73.276434,-151.097567,-1,-1,2020.21,21,unassigned
2,A/swine/Tennessee/A02428125/2018,103.878946,134.591555,-1,-1,2018.97,19,unassigned
3,A/Shiraz/1006/2015,64.402457,89.854161,-1,-1,2015.81,22,unassigned
4,A/Alabama/09/2015,3.497593,0.332706,1,19,2015.12,174,3c2.A
...,...,...,...,...,...,...,...,...
2593,A/swine/Nebraska/A02256553/2018,95.949991,197.170133,-1,-1,2018.62,7,unassigned
2594,A/swine/Minnesota/A02245583/2020,220.937493,1.940319,-1,-1,2020.27,4,unassigned
2595,A/swine/BritishColumbia/SD0286/2018,196.087181,-100.649552,-1,-1,2018.27,2,unassigned
2596,A/swine/Saskatchewan/SD0271/2018,160.470705,140.218308,-1,-1,2018.18,3,unassigned


In [94]:
domain =  merged_mds_df["mds_label"].drop_duplicates().values
range_ = colors[len(domain)-1:len(domain)].dropna(axis=1).values.tolist()[0]
chart_12_mds = scatterplot_with_tooltip_interactive(merged_mds_df,'mds1','mds2','mds1','mds2',['strain','clade_membership', 'mds_label'],'mds_label:N', domain, range_).properties(title="mds, optimal distance threshold")
domain = merged_mds_df["mds_label_default"].drop_duplicates().values
range_ = colors[len(domain)-1:len(domain)].dropna(axis=1).values.tolist()[0]
chart_12_default = scatterplot_with_tooltip_interactive(merged_mds_df,'mds1','mds2','mds1','mds2',['strain','clade_membership', 'mds_label', 'mds_label_default'],'mds_label_default:N', domain, range_).properties(title="mds, default distance threshold")
domain = merged_mds_df["clade_membership"].drop_duplicates().values
range_ = colors[len(domain)-1:len(domain)].dropna(axis=1).values.tolist()[0]
chart_12_clade = scatterplot_with_tooltip_interactive(merged_mds_df,'mds1','mds2','mds1','mds2',['strain','clade_membership', 'mds_label'],'clade_membership:N', domain, range_).properties(title="mds, clade membership")
charts = alt.hconcat(chart_12_mds,chart_12_default).resolve_scale(color='independent')
charts
#fullChart.save(snakemake.output.SupplementalmdsHDBSCAN)
#save(fullChart, snakemake.output.SupplementalmdsHDBSCANPNG, scale_factor=2.0)

alt.HConcatChart(...)

In [95]:
from Helpers import get_euclidean_data_frame
from sklearn.metrics import confusion_matrix, matthews_corrcoef

In [96]:
def _get_embedding_columns_by_method(method):
    if method in ("pca"):
        return list(f"{method}1 {method}2 {method}3 {method}4 {method}5 {method}6 {method}7 {method}8 {method}9 {method}10".split())
    if method in ("mds"):
        return list(f"{method}1 {method}2".split())
    if method in ("mds"):
        return list("mds1 mds2".split())
    else:
        return list(f"{method}_x {method}_y".split())

In [98]:
MCC_val = []

for method in ["mds"]:
    KDE_df_normal = get_euclidean_data_frame(sampled_df=merged_mds_df, column_for_analysis="clade_membership", embedding="method", column_list=_get_embedding_columns_by_method(method))

    KDE_df_cluster =  get_euclidean_data_frame(sampled_df=merged_mds_df, column_for_analysis=f"{method}_label", embedding="method", column_list=_get_embedding_columns_by_method(method))

    confusion_matrix_val = confusion_matrix(KDE_df_normal["clade_status"], KDE_df_cluster["clade_status"])
    matthews_cc_val = matthews_corrcoef(KDE_df_normal["clade_status"], KDE_df_cluster["clade_status"])
    MCC_val.append(matthews_cc_val)

In [127]:
domain =  sorted(merged_mds_df["mds_label"].drop_duplicates().values)
range_ = ["#999999"] + colors[len(domain)-1:len(domain)].dropna(axis=1).values.tolist()[0]
MDS = linking_tree_with_plots_brush(
    merged_mds_df,
    ['mds1', 'mds2'],
    ['MDS1', 'MDS2'],
    'mds_label:N',
    ['strain','clade_membership'],
    domain,
    range_
)
domain =  sorted(merged_mds_df["mds_label_default"].drop_duplicates().values)
range_ = ["#999999"] + colors[len(domain)-1:len(domain)].dropna(axis=1).values.tolist()[0]
mds = linking_tree_with_plots_brush(
    merged_mds_df,
    ['mds1', 'mds2'],
    ['mds1', 'mds2'],
    'mds_label_default:N',
    ['strain','clade_membership'],
    domain,
    range_
)

In [128]:
MDSChart = MDS[0]|MDS[1].properties(title= "MCC Value: " + str(round(MCC_val[0],3)))
MDSChartDefault = mds[0]|mds[1].properties(title= "Default. MCC Value: " + str(round(MCC_val[0],3)))

In [129]:
alt.vconcat(MDSChartDefault, MDSChart).resolve_scale(color='independent')

alt.VConcatChart(...)

In [164]:
MDS_df_full

,strain,mds1,mds2,mds_label,outlier_status
0,A/Netherlands/179/1993,-59.616050,107.664263,2,valid
1,A/HongKong/2/1994,-64.057328,109.818813,2,valid
2,A/Lyon/1815/1993,-62.701057,109.484116,2,valid
3,A/Lyon/23602/1993,-61.420779,109.115020,2,valid
4,A/Netherlands/372/1993,-61.036153,110.580199,2,valid
...,...,...,...,...,...
5861,A/Maine/06/2011,48.109427,160.063177,-1,outlier
5862,A/environment/Indiana/16TOSU3546/2016,72.498844,167.352855,-1,outlier
5863,A/Kansas/13/2009,37.573574,150.969532,-1,outlier
5864,A/Guangdong/YueFang277/2017,90.821299,59.725127,-1,outlier


In [167]:
MDS_df_full["outlier_status"] = MDS_df_full["mds_label"].apply(lambda label: "outlier" if label==-1 else "not_outlier")

In [171]:
domain = sorted(list(set(MDS_df_full["outlier_status"].values)))
range_ = ["#999999"] + colors[len(domain)-1:len(domain)].dropna(axis=1).values.tolist()[0]

In [172]:
domain

['outlier', 'valid']

In [173]:
alt.data_transformers.disable_max_rows()
list_of_chart = scatterplot_with_tooltip_interactive(MDS_df_full, 'mds1','mds2',"MDS1", "MDS2", 'outlier_status:N', "outlier_status:N", domain, range_)
list_of_chart

alt.Chart(...)

In [183]:
with open("config/outliers_h3n2_full.txt") as w:
    outliers = [line.strip() for line in w]

In [185]:
MDS_df_human_swine["isin_outliers"] = MDS_df_human_swine["strain"].isin(outliers)

In [186]:
MDS_df_human_swine["isin_outliers"].value_counts()

False    2593
True        5
Name: isin_outliers, dtype: int64

In [187]:
MDS_df_human_swine.query("mds_label==-1")["isin_outliers"].value_counts()

False    122
True       5
Name: isin_outliers, dtype: int64

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()
chart1 = alt.Chart(full_outlier).mark_circle(size=60).encode(
            x=alt.X('X_scores', title="X_scores"),
            color='predicted_outlier_status:N',
            tooltip=['strain']
        )
chart1

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 2))
ax = sns.kdeplot(human_swine_outlier["X_scores"], ax=ax)
ax.axvline(x=human_swine_outlier["X_scores"].mean() - 4*human_swine_outlier["X_scores"].std(), color="red")
ax = sns.rugplot(data=human_swine_outlier, x="X_scores", color="orange", ax=ax, height=.1)
ax.set(xlim=(-20, -.5))
ax.set(xlabel='Local Outlier Factor Scores', ylabel='Density')
plt.savefig(snakemake.output.distributionHumanSwine)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 2))
ax = sns.kdeplot(full_outlier["X_scores"], ax=ax)
ax.axvline(x=full_outlier["X_scores"].mean() - 4*full_outlier["X_scores"].std(), color="red")
ax = sns.rugplot(data=full_outlier, x="X_scores", color="orange", ax=ax, height=.1)
ax.set(xlim=(-20, -.5))
ax.set(xlabel='Local Outlier Factor Scores', ylabel='Density')
plt.savefig(snakemake.output.distributionFull)

In [ ]:
MDS_KDE_df_human_swine = pd.read_csv("results/KDEDensity_mds_human_swine.csv",index_col=0)
MDS_KDE_metadata_df_human_swine = pd.read_csv("results/KDEDensity_mds_metadata_human_swine.csv", index_col=0)
MDS_outlier_metadata_df_human_swine = pd.read_csv("../docs/LOF_accuracy_human_swine.csv")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 3))

ax = sns.kdeplot(MDS_KDE_df_human_swine.query("clade_status == 1")["scaled_distance"], label="Same clade", ax=ax)
ax = sns.kdeplot(MDS_KDE_df_human_swine.query("clade_status == 0")["scaled_distance"], label="Different clade", ax=ax)

ax.axvline(x=MDS_KDE_metadata_df_human_swine["threshold"].values.tolist()[0], label="SVC threshold", color="#000000", alpha=0.5)
ax.axvline(x=MDS_outlier_metadata_df_human_swine["threshold"].values.tolist()[0], label="Outlier threshold", color="#800000", alpha=0.5)

ax.set_xlabel("Scaled Euclidean distance ")
ax.set_ylabel("KDE density")

fig.suptitle('MDS KDE Plot', fontsize=16)
sns.despine()
plt.savefig(snakemake.output.KDE_densityHumanSwine)